<a href="https://colab.research.google.com/github/vvamsi91/IML_AS6/blob/main/IML_AS6_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

!pip install ipython-autotime
%load_ext autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.0 MB/s eta 0:00:00
time: 339 µs (started: 2023-12-02 00:29:02 +00:00)


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

time: 10.9 s (started: 2023-12-02 00:29:53 +00:00)


In [3]:
torch.manual_seed(123)

time: 7.04 ms (started: 2023-12-02 00:31:49 +00:00)


In [4]:
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
mean, std = imgs.view(3, -1).mean(dim=1), imgs.view(3, -1).std(dim=1)


100%|██████████| 170498071/170498071 [00:02<00:00, 73376949.28it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
time: 25.3 s (started: 2023-12-02 00:34:58 +00:00)


In [5]:

mean



tensor([0.4914, 0.4822, 0.4465])

time: 40.2 ms (started: 2023-12-02 00:37:14 +00:00)


In [6]:

std

tensor([0.2470, 0.2435, 0.2616])

time: 19.3 ms (started: 2023-12-02 00:41:09 +00:00)


In [14]:
#Setting Device
Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Device



device(type='cpu')

time: 4.7 ms (started: 2023-12-02 00:47:49 +00:00)


In [15]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))])


time: 928 µs (started: 2023-12-02 00:47:51 +00:00)


In [16]:
cifar10 = datasets.CIFAR10('./data', train=True, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))


time: 624 ms (started: 2023-12-02 00:49:18 +00:00)


In [17]:
cifar10_val = datasets.CIFAR10('./data', train=False, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2470, 0.2435, 0.2616))]))


time: 497 ms (started: 2023-12-02 00:50:08 +00:00)


In [18]:

first_image, label = cifar10[0]
print(first_image.shape)


torch.Size([3, 32, 32])
time: 12.7 ms (started: 2023-12-02 00:51:51 +00:00)


In [19]:
train_loader = DataLoader(cifar10, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(cifar10_val, batch_size=32, shuffle=False, num_workers=2)


time: 1.4 ms (started: 2023-12-02 00:54:10 +00:00)


In [20]:
model = nn.Sequential(nn.Flatten(), nn.Linear(32 * 32 * 3, 512), nn.Tanh(), nn.Linear(512, 10)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)


time: 111 ms (started: 2023-12-02 00:55:00 +00:00)


In [21]:

def train_model(model, train_loader, test_loader, num_epochs=300, lr=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Testing the model
        model.eval()
        correct = 0
        total = 0
        all_predicted = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                all_predicted.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy = correct / total

        if epoch % 1 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}, Test Accuracy: {accuracy * 100:.2f}%')

    # Classification Report
    report = classification_report(all_labels, all_predicted)
    print("Classification Report:\n", report)


time: 2.98 ms (started: 2023-12-02 00:55:33 +00:00)


In [22]:

train_model(model, train_loader, test_loader, num_epochs=300, lr=0.01)

Epoch 1/300, Loss: 1.7886348081870638, Test Accuracy: 40.94%
Epoch 2/300, Loss: 1.6521172705020037, Test Accuracy: 43.15%
Epoch 3/300, Loss: 1.5804378465437692, Test Accuracy: 44.45%
Epoch 4/300, Loss: 1.5198366514246813, Test Accuracy: 45.26%
Epoch 5/300, Loss: 1.4625313013544161, Test Accuracy: 46.44%
Epoch 6/300, Loss: 1.4085316710646, Test Accuracy: 47.50%
Epoch 7/300, Loss: 1.3539841107580803, Test Accuracy: 47.11%
Epoch 8/300, Loss: 1.3031534532744078, Test Accuracy: 47.23%
Epoch 9/300, Loss: 1.249340126015632, Test Accuracy: 48.36%
Epoch 10/300, Loss: 1.1990976107097633, Test Accuracy: 47.90%
Epoch 11/300, Loss: 1.147438968516891, Test Accuracy: 48.39%
Epoch 12/300, Loss: 1.0958708692496966, Test Accuracy: 49.10%
Epoch 13/300, Loss: 1.0460041413990564, Test Accuracy: 48.73%
Epoch 14/300, Loss: 0.9943198018247931, Test Accuracy: 48.12%
Epoch 15/300, Loss: 0.9466876947223873, Test Accuracy: 48.06%
Epoch 16/300, Loss: 0.8974496146195681, Test Accuracy: 48.03%
Epoch 17/300, Loss: 0.

In [24]:
model2 = nn.Sequential(nn.Flatten(), nn.Linear(32 * 32 * 3, 512), nn.Tanh(), nn.Linear(512, 256), nn.Tanh(), nn.Linear(256, 128), nn.Tanh(), nn.Linear(128, 10)).to(device)


time: 31.7 ms (started: 2023-12-02 03:49:59 +00:00)


In [25]:
train_model(model2, train_loader, test_loader, num_epochs=120, lr=0.01)

Epoch 1/120, Loss: 1.8600392635251495, Test Accuracy: 38.96%
Epoch 2/120, Loss: 1.6882640814369334, Test Accuracy: 42.43%
Epoch 3/120, Loss: 1.6143167977979835, Test Accuracy: 43.48%
Epoch 4/120, Loss: 1.551621328159852, Test Accuracy: 44.71%
Epoch 5/120, Loss: 1.4958151898853915, Test Accuracy: 46.19%
Epoch 6/120, Loss: 1.4415796293101857, Test Accuracy: 47.23%
Epoch 7/120, Loss: 1.38489851921854, Test Accuracy: 47.29%
Epoch 8/120, Loss: 1.3294328470605348, Test Accuracy: 46.58%
Epoch 9/120, Loss: 1.2698992878599038, Test Accuracy: 47.04%
Epoch 10/120, Loss: 1.2136496130427106, Test Accuracy: 46.41%
Epoch 11/120, Loss: 1.1531435336276499, Test Accuracy: 46.54%
Epoch 12/120, Loss: 1.092544283915695, Test Accuracy: 48.14%
Epoch 13/120, Loss: 1.0330894368433143, Test Accuracy: 47.67%
Epoch 14/120, Loss: 0.9694620635520169, Test Accuracy: 46.01%
Epoch 15/120, Loss: 0.9129247236389116, Test Accuracy: 47.83%
Epoch 16/120, Loss: 0.8515014192154983, Test Accuracy: 47.11%
Epoch 17/120, Loss: 0